## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [2]:
import pandas as pd
data_path = '../data/cookie_cats.csv'
df = pd.read_csv(data_path)

In [3]:
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [4]:
mean_retention_7 = df.groupby('version')['retention_7'].mean()
print(mean_retention_7)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


Виходячи з середнього значення краще показник повернення через 7 днів у контрольної групи з воротами на 30 рівні

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [7]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Розділення даних на контрольну і тестову групи
control_group = df[df['version'] == 'gate_30']
treatment_group = df[df['version'] == 'gate_40']

In [29]:
control_group

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
6,1066,gate_30,0,False,False
11,2101,gate_30,0,False,False
13,2179,gate_30,39,True,False
...,...,...,...,...,...
90179,9998576,gate_30,14,True,False
90180,9998623,gate_30,7,False,False
90182,9999178,gate_30,21,True,False
90183,9999349,gate_30,10,False,False


In [33]:
# 
control_retained = control_group['retention_7'].sum()
control_total = control_group['retention_7'].count()
treatment_retained = treatment_group['retention_7'].sum()
treatment_total = treatment_group['retention_7'].count()

In [30]:
print(control_total)

44700


In [15]:
# Знаходмо пропорції
control_rate = control_retained / control_total
treatment_rate = treatment_retained / treatment_total

print(control_rate, treatment_rate)

0.19020134228187918 0.18200004396667327


In [34]:
# Виконуємо Z-test
count = np.array([control_retained, treatment_retained])
nobs = np.array([control_total, treatment_total])
z_stat, p_val = proportions_ztest(count, nobs)

print(f"z statistic: {z_stat:}")
print(f"p-value: {p_val:}")

z statistic: 3.164358912748191
p-value: 0.001554249975614329


In [19]:
# Розраховуємо довірчий інтревал
confint_control = proportion_confint(control_retained, control_total, alpha=0.05)
confint_treatment = proportion_confint(treatment_retained, treatment_total, alpha=0.05)


print(f"Довірчий інтервал 95% для групи control: [{confint_control[0]:.4f}, {confint_control[1]:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment [{confint_treatment[0]:.4f}, {confint_treatment[1]:.4f}]")



Довірчий інтервал 95% для групи control: [0.1866, 0.1938]
Довірчий інтервал 95% для групи treatment [0.1785, 0.1855]


p-value має значення менше ніж 0.05 тож відхилеяєму нульову гіпотезу, є статистична значущість між версіями гри та поверненням юзерів на 7 день 
Також значення довірчих інтервалів не пересікаються шо також свідчить про статичтичну значушость розподілу вибірок і для групи control вони вище ніж у treatment шо свідчить про перевагу версіі з воротами на 30 день  

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [22]:
contingency_table = pd.crosstab(df['version'], df['retention_7'])
contingency_table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [35]:

import scipy.stats as stats

# Виконуємо тест хі-квадрат незалежності
chi2_stat, p_val, dof, ex = stats.chi2_contingency(contingency_table)

# Виводимо результати
print(f"Chi-squared statistic: {chi2_stat:}")
print(f"p-value: {p_val:}")


# Інтерпретація результатів
alpha = 0.05
if p_val < alpha:
    print("Відхиляємо нульову гіпотезу. Є статистично значуща залежність між версією гри і значенням retention_7.")
else:
    print("Не можемо відхилити нульову гіпотезу. Немає статистично значущої залежності між версією гри і значенням retention_7.")

Chi-squared statistic: 9.959086799559167
p-value: 0.0016005742679058301
Відхиляємо нульову гіпотезу. Є статистично значуща залежність між версією гри і значенням retention_7.
